In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense 
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
from PIL import Image
from tqdm import tqdm
import os 
import time
import matplotlib.pyplot as plt

In [2]:
translate_list = [' ',
             '1',
             '2',
             '3',
             '4',
             '5',
             '6',
             '7',
             '8',
             '9',
             '0',
             '+',
             '-',
             '*',
             '/',
             '=',
            ]

In [3]:
CHAR_LIST_LEN = len(translate_list)
PROBLEM_LENGTH = 20
PROBLEM_DIM = PROBLEM_LENGTH * CHAR_LIST_LEN

SEED_SIZE = 100

N_EPOCHS = 10
BATCH_SIZE = 5
BUFFER_SIZE = 60000

In [4]:
def prob_to_array(in_string):
    problem_array = []
    for char in in_string:
        value = translate_list.index(char)
        char_array = [0]*len(translate_list)
        char_array[value] = 1
        problem_array.append(char_array)
    if len(in_string) < PROBLEM_LENGTH:
        for _ in range(PROBLEM_LENGTH-len(in_string)):
            char_array = [0]*len(translate_list)
            char_array[0] = 1
            problem_array.append(char_array)
    return problem_array

def array_to_prob(problem_array):
    problem_string = ""
    for char_array in problem_array:
        largest = max(char_array)
        value = char_array.index(largest)
        char = translate_list[value]
        problem_string += char
    problem_string.strip()
    return problem_string

def np_array_to_prob(problem_array):
    problem_string = ""
    for char_array in problem_array:
        value = np.max(char_array)
        print(value)
        char = translate_list[value]
        problem_string += char
    problem_string.strip()
    return problem_string



#myarr = prob_to_array('90+21')
#myprob = array_to_prob(myarr)
#print(myarr)
#print(myprob)


In [5]:
import random
def generate_input(count):
    output_list = []
    for _ in range(count):
        left = random.randint(1,99)
        right = random.randint(1,99)
        problem = str(left) + '+' + str(right)
        output_list.append(problem)
    return output_list

In [6]:
input_problems = generate_input(1000)
# print(input_problems)
training_data = []
for item in input_problems:
    temp_arr = prob_to_array(item)
    training_data.append(temp_arr)
training_data = np.asarray(training_data)
training_data = training_data.astype(np.float32)
print("Shape: " + str(training_data.shape))

Shape: (1000, 20, 16)


In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices(training_data) \
    .shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [8]:
generator = Sequential([
    Dense(64,activation="relu",input_dim=100),
    Dense(128, activation="selu"),
    #model.add(Dense(28*28, activation="sigmoid")),
    Dense(20*16, activation="sigmoid"),
    Reshape([20, 16])
])

discriminator = Sequential([
    Flatten(input_shape=[20, 16]),
    Dense(128, activation="selu"),
    Dense(64, activation="selu"),
    Dense(1, activation="sigmoid")
])

In [9]:
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
gan = Sequential([generator, discriminator])
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [10]:
def display_problems(fake_problems):
    numpy_array = fake_problems.numpy()
    #print(numpy_array)
    py_array = numpy_array.tolist()
    for prob_array in py_array:
        # print(prob_array)
        prob_string = array_to_prob(prob_array)
        print(prob_string)
    # print(fake_problems.shape)

In [11]:
"""
What I've noticed is that the output arrays do differ.
The 0 index of the first 5 characters is always very small
    0 index of last 15 are equal to 0 whereas all other characters equal a small number
Indexes of 1 to + are all equal to 1.0 for the first two characters
The array of the third character can be:
[8.040736530136039e-18, 1.0603317252406942e-16, 1.0, 1.0, 1.0, 1.834101947385663e-17, 1.0, 1.0, 1.4876883021435605e-17, 1.6702045462505285e-17, 7.937368401196717e-17, 1.0, 6.892058275879986e-16, 3.727383787233881e-18, 2.180618824768212e-17, 2.4552828236774647e-17]
Is the probability maxed out at 1 or something?
    Maybe the program rounded down from a higher probability to just 1
Problem is most likely in the generator model.
"""
random_normal_dimensions = 100
problem_count = 0
for epoch in range(N_EPOCHS):
    for real_problems in train_dataset:
        print("------------------------------")
        problem_count += 1
        print("Problem Count: " + str(problem_count))
        # print("Real Problems: " + str(real_problems.shape))
        noise = tf.random.normal(shape=[BATCH_SIZE, random_normal_dimensions])
        # print("Noise: " + str(noise.shape))
        fake_problems = generator(noise)
        print("Real: ")
        display_problems(real_problems)
        print("Fakes: ")
        display_problems(fake_problems)
        mixed_problems = tf.concat([fake_problems, real_problems], axis=0)
        # print("Mixed problems: " + str(mixed_problems.shape))
        discriminator_labels = tf.constant([[0.]] * BATCH_SIZE + [[1.]] * BATCH_SIZE)
        # print("Discrim labels: " + str(discriminator_labels.shape))
        discriminator.trainable = True
        print("Discriminator loss: " + str(discriminator.train_on_batch(mixed_problems, discriminator_labels)))
        #print("Discriminator summary: " + str(discriminator.summary()))
        noise = tf.random.normal(shape=[BATCH_SIZE, random_normal_dimensions])
        generator_labels = tf.constant([[1.]] * BATCH_SIZE)
        discriminator.trainable = False
        print("Generator loss: " + str(gan.train_on_batch(noise, generator_labels)))
        # print("GAN summary: " + str(gan.summary()))

------------------------------
Problem Count: 1
Real: 
44+55               
41+35               
47+40               
86+90               
73+28               
Fakes: 
3/405 206**365-22602
303748/1*6/360=3927=
321+-4216493+=22/652
0+3/4/4+6*948759*==9
0=3070086/ 58*=08=60
Discriminator loss: 0.6990314722061157
Generator loss: 8.18448257446289
------------------------------
Problem Count: 2
Real: 
9+19                
3+93                
6+8                 
46+71               
13+92               
Fakes: 
 /+5*31* /918==2++* 
-437*240+*9+8523 -53
6/3758/*65=367=3/302
3-3*98/3*=*460=3-626
141542/*24534=26/*04
Discriminator loss: 1.257460117340088
Generator loss: 4.834436893463135
------------------------------
Problem Count: 3
Real: 
35+98               
1+19                
13+74               
81+40               
59+63               
Fakes: 
 /304/30*69389-24602
1 100+3+/*9+4 2 1=02
18150+30+* +89-8-392
 314*040+*336  81-94
 /75*3306=9865=2/162
Discriminator loss: 0.849851965904235

------------------------------
Problem Count: 40
Real: 
5+92                
54+98               
62+13               
92+48               
88+99               
Fakes: 
2 ==003 / 7+70= /  3
7 ==003 / 7+7 = /  3
2 ==003 /87+79= /  -
1 ==003 /83+79= / *3
2 ==0 3  87470= / *3
Discriminator loss: 0.8917025327682495
Generator loss: 0.5688592195510864
------------------------------
Problem Count: 41
Real: 
75+76               
2+93                
16+11               
34+11               
26+62               
Fakes: 
2 =5003 /83+79= /  3
2 ==003 /83+79= /7*3
2 =5003 /83+79= /  +
2 ==003 /87+79= /7 3
2 ==003 /87+7 = / *+
Discriminator loss: 0.8454376459121704
Generator loss: 0.5265196561813354
------------------------------
Problem Count: 42
Real: 
84+67               
75+98               
21+6                
93+20               
12+63               
Fakes: 
2 ==003  87+79= /  3
7 ==0 3 /83+7 = /  +
2 ==0-3  87+79= /  3
2 ===03  85+ 9= 1  3
2 =5003  87+79= /7 3
Discriminator loss: 0.93711662

Generator loss: 0.4714359641075134
------------------------------
Problem Count: 84
Real: 
32+63               
22+78               
72+19               
56+36               
62+38               
Fakes: 
7 ==--3 / 7  97 /  -
7 ==0-3   7  97 /  -
7 ==--3   7+ 97 1  -
7 ==--3 / 7+ 97 /  -
7 ==--3  87  97 /  -
Discriminator loss: 0.8902945518493652
Generator loss: 0.4544396996498108
------------------------------
Problem Count: 85
Real: 
36+35               
48+27               
20+19               
33+48               
89+30               
Fakes: 
7 ==--3 / 7+ 97    -
7 =4--3 /87  97    -
7 ==--3   7  97 1  -
7 ==--3   7  97    -
7 ==--3   7  97 /  -
Discriminator loss: 0.9010927081108093
Generator loss: 0.4594975411891937
------------------------------
Problem Count: 86
Real: 
83+26               
16+24               
57+56               
86+18               
2+86                
Fakes: 
7 ==--3  873 97 /  -
7 ==--3   7  97 /  -
7 ==--3   7  97 /  -
7 ==--3 / 7  97 /  -
7 =4--3  87  97 

Generator loss: 0.3089674115180969
------------------------------
Problem Count: 127
Real: 
45+22               
16+8                
15+56               
58+89               
37+99               
Fakes: 
4 =8--  /    9      
4 =8--  /    9      
4 =8--  /    9      
4 =8--  /    9      
4 =8--  /    9      
Discriminator loss: 1.2179502248764038
Generator loss: 0.2945994436740875
------------------------------
Problem Count: 128
Real: 
14+94               
24+62               
88+55               
29+99               
61+80               
Fakes: 
4 =8--  /    9      
4 =8--  /    9      
4 =8--  /    9      
4 =8--  /    9      
4 =8--  /    9      
Discriminator loss: 1.2471727132797241
Generator loss: 0.2996300756931305
------------------------------
Problem Count: 129
Real: 
96+15               
14+85               
2+38                
49+48               
85+10               
Fakes: 
4 =8--  /    9      
4 =8--  /    9      
4 =8--  /    9      
4 =8--  /    9      
4 =8--  /    

Generator loss: 0.19433529675006866
------------------------------
Problem Count: 171
Real: 
64+75               
82+12               
70+76               
49+16               
98+6                
Fakes: 
8+464               
4+464               
89467               
8+464               
4+467               
Discriminator loss: 1.4217621088027954
Generator loss: 0.1931227147579193
------------------------------
Problem Count: 172
Real: 
12+54               
78+71               
70+94               
70+45               
94+91               
Fakes: 
8+467               
4+467               
4+464               
8+464               
8+464               
Discriminator loss: 1.4192566871643066
Generator loss: 0.1984669268131256
------------------------------
Problem Count: 173
Real: 
90+21               
22+92               
18+21               
61+55               
7+38                
Fakes: 
8+469               
8+464               
4+467               
4+467               
8+464       

...

111                 
111                 
Discriminator loss: 1.6609628200531006
Generator loss: 0.09191172569990158
------------------------------
Problem Count: 1821
Real: 
3+22                
77+75               
56+70               
62+8                
45+22               
Fakes: 
111                 
111                 
111                 
111                 
111                 
Discriminator loss: 1.6609792709350586
Generator loss: 0.09026734530925751
------------------------------
Problem Count: 1822
Real: 
83+61               
78+55               
29+17               
61+14               
70+94               
Fakes: 
111                 
111                 
111                 
111                 
111                 
Discriminator loss: 1.6551603078842163
Generator loss: 0.09097637236118317
------------------------------
Problem Count: 1823
Real: 
43+74               
52+24               
14+96               
61+74               
53+97               
Fakes: 
111       

32+11               
63+44               
85+98               
50+69               
15+57               
Fakes: 
111                 
111                 
111  8              
111                 
111                 
Discriminator loss: 1.6613041162490845
Generator loss: 0.0911790207028389
------------------------------
Problem Count: 1864
Real: 
33+10               
45+71               
63+11               
81+40               
34+11               
Fakes: 
111                 
111                 
111                 
111                 
111                 
Discriminator loss: 1.662306547164917
Generator loss: 0.09045219421386719
------------------------------
Problem Count: 1865
Real: 
18+2                
58+26               
63+88               
45+98               
46+92               
Fakes: 
111                 
111                 
111                 
111                 
111                 
Discriminator loss: 1.6612637042999268
Generator loss: 0.09058544039726257
-------

Discriminator loss: 1.6618493795394897
Generator loss: 0.0901356115937233
------------------------------
Problem Count: 1907
Real: 
35+56               
34+76               
66+21               
36+87               
21+55               
Fakes: 
111                 
111                 
111                 
111                 
111                 
Discriminator loss: 1.664097785949707
Generator loss: 0.0897158533334732
------------------------------
Problem Count: 1908
Real: 
79+4                
24+1                
33+41               
72+53               
38+69               
Fakes: 
111                 
111                 
111                 
111                 
111                 
Discriminator loss: 1.6648094654083252
Generator loss: 0.09006063640117645
------------------------------
Problem Count: 1909
Real: 
42+14               
84+44               
68+20               
23+69               
57+28               
Fakes: 
111                 
111                 
111          

------------------------------
Problem Count: 1949
Real: 
44+78               
95+62               
99+39               
49+76               
82+13               
Fakes: 
111                 
111                 
111                 
111                 
111                 
Discriminator loss: 1.6644971370697021
Generator loss: 0.08947888761758804
------------------------------
Problem Count: 1950
Real: 
48+29               
14+85               
73+7                
55+50               
47+32               
Fakes: 
111                 
111                 
111                 
111                 
111                 
Discriminator loss: 1.6652113199234009
Generator loss: 0.08980010449886322
------------------------------
Problem Count: 1951
Real: 
90+21               
58+25               
6+42                
20+58               
86+99               
Fakes: 
111                 
111                 
111                 
111                 
111                 
Discriminator loss: 1.

Generator loss: 0.08902039378881454
------------------------------
Problem Count: 1993
Real: 
79+98               
18+37               
28+67               
46+74               
77+89               
Fakes: 
111                 
111                 
111                 
111                 
111                 
Discriminator loss: 1.6665624380111694
Generator loss: 0.0894797146320343
------------------------------
Problem Count: 1994
Real: 
34+47               
86+51               
89+4                
63+30               
93+35               
Fakes: 
111                 
111                 
111                 
111                 
111                 
Discriminator loss: 1.6658369302749634
Generator loss: 0.08945301920175552
------------------------------
Problem Count: 1995
Real: 
94+56               
86+90               
96+21               
47+43               
29+82               
Fakes: 
111                 
111                 
111                 
111                 
111     